In [78]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [79]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import cv2
import PIL.Image
import time
import os
import functools

In [80]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [81]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [82]:
target_path = '/content/gdrive/MyDrive/Colab Notebooks/Projects/images/targets/'
input_path = '/content/gdrive/MyDrive/Colab Notebooks/Projects/images/inputs/'

In [83]:
target_imgs = os.listdir(target_path)
input_imgs = os.listdir(input_path)

print(f"Total target images: {len(target_imgs)}")
print(f"Total input images: {len(input_imgs)}")

Total target images: 7
Total input images: 1


## Visualize the input

Define a function to load an image and limit its maximum dimension to 512 pixels.

In [84]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

Create a simple function to display an image:

In [85]:
def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)

In [86]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
content_layers = ['block5_conv2']

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

In [87]:
def vgg_layers(layer_names):
    """ Creates a vgg model that returns a list of intermediate output values."""
    # Load our model. Load pretrained VGG, trained on imagenet data
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    outputs = [vgg.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([vgg.input], outputs)
    return model

In [88]:
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

In [89]:
class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg =  vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs*255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                        outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output)
                        for style_output in style_outputs]

        content_dict = {content_name:value 
                        for content_name, value 
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name:value
                    for style_name, value
                    in zip(self.style_layers, style_outputs)}
        
        return {'content':content_dict, 'style':style_dict}

In [90]:
def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [91]:
def style_content_loss(outputs):
        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                            for name in style_outputs.keys()])
        style_loss *= style_weight / num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                                for name in content_outputs.keys()])
        content_loss *= content_weight / num_content_layers
        loss = style_loss + content_loss
        return loss

In [92]:
def high_pass_x_y(image):
    x_var = image[:,:,1:,:] - image[:,:,:-1,:]
    y_var = image[:,1:,:,:] - image[:,:-1,:,:]

    return x_var, y_var

In [93]:
def total_variation_loss(image):
    x_deltas, y_deltas = high_pass_x_y(image)
    return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

In [94]:
style_extractor = vgg_layers(style_layers)
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
style_weight=1e-2
content_weight=1e4
total_variation_weight=30
output_path = "/content/gdrive/MyDrive/Colab Notebooks/Projects/images/outputs"

In [95]:
for target_img in target_imgs:
    @tf.function()
    def train_step(image):
        with tf.GradientTape() as tape:
            outputs = extractor(image)
            loss = style_content_loss(outputs)
            loss += total_variation_weight*tf.image.total_variation(image)

        grad = tape.gradient(loss, image)
        opt.apply_gradients([(grad, image)])
        image.assign(clip_0_1(image))

    target_name = target_img[:target_img.index('.')]
    style_img = load_img(f"{target_path}{target_img}")
    for input_img in input_imgs:
        # Show the image that will be proccessed
        content_img = load_img(f"{input_path}{input_img}")

        # Extract whats needed
        style_outputs = style_extractor(style_img*255)
        extractor = StyleContentModel(style_layers, content_layers)

        # Running gradient descent
        style_targets = extractor(style_img)['style']
        content_targets = extractor(content_img)['content']

        image = tf.Variable(content_img)
        start = time.time()

        epochs = 10
        steps_per_epoch = 300

        step = 0
        content_name = input_img[:input_img.index('.')]
        # !mkdir "{output_path}/process/{content_name}/"
        count = 1
        process_imgs = []
        for n in range(epochs):
            print("="*15, target_name, "="*15)
            print("\t","="*10, content_name, "="*10)
            for m in range(steps_per_epoch):
                step += 1
                train_step(image)
                print(".", end='')
                ep_result = tensor_to_image(image)
                # ep_result.save(f"{output_path}/process/{content_name}_{target_name}/{count}.png")
                process_imgs.append(ep_result)
                count += 1
            display.clear_output(wait=True)
            print("\t\tTrain step: {}".format(step))
        print("\t\t Saving file...")
        process_imgs[0].save(fp=f"{output_path}/process/{content_name}_{target_name}.gif", save_all=True, format='GIF', duration=1, loop=0, append_images=process_imgs[1:])
        end = time.time()
        print("\t\tTotal time: {:.1f}".format(end-start))
        # Save the image to the google drive
        style_result = tensor_to_image(image)
        style_result.save(f"{output_path}/stylized/{content_name}_{target_name}.png")

		Train step: 3000
		 Saving file...
		Total time: 298.6
